In [2]:
import requests
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from scipy import fft, arange
import json
import pyrebase

In [3]:
config = json.load(open('./private/config.json'))

firebase = pyrebase.initialize_app(config)
db = firebase.database()


def pushtoDB(payload):
	payloadKey = list(payload)[0]
	data = payload[payloadKey] 
	db.child(payloadKey).set(data)
	return  json.dumps({"success": 1})

def getDB():
	return db.get().val()

def deleteAll():
	db.set({})
	return json.dumps({"success": 1})

In [4]:
r = getDB()
r = r['data']

In [5]:
df =pd.DataFrame()
y=[]
for key in r:
    y.append(r[key]['prediction'])
    x = []
    for sample in r[key]['datapoints']:
        x.append(sample['ax'])
        x.append(sample['ay'])
        x.append(sample['az'])
        x.append(sample['gx'])
        x.append(sample['gy'])
        x.append(sample['gz'])
    df=pd.concat([df,pd.DataFrame(x).transpose()])

df.reset_index(drop=True, inplace=True) 
df = df[df.columns[0:120]]
df.fillna(value=0, inplace=True)
# take only first 20 columns
y = pd.get_dummies(y)
ymap = y.columns

In [6]:
df

,0,1,2,3,4,5,6,7,8,9,...,110,111,112,113,114,115,116,117,118,119
0,0.148972,-0.272308,-0.907806,-0.329643,-0.348913,-0.151315,0.143143,-0.273575,-0.961304,-1.826095,...,-1.263458,0.189870,0.299546,0.088445,-0.093903,-0.143906,-1.041748,0.234859,0.131649,-0.044328
1,0.089371,-0.186249,-0.972000,-0.063619,-0.153382,-0.071475,0.087524,-0.196747,-1.012772,-0.224193,...,-1.156235,0.390914,0.240631,0.062555,-0.016006,-0.149261,-0.976929,-0.287814,-0.319969,-0.329707
2,-0.151413,-0.283417,-0.977158,0.035538,-0.013706,-0.021875,-0.153519,-0.278305,-0.950363,-0.064154,...,-0.965286,-0.078880,0.133709,0.200437,-0.121689,-0.229477,-0.970551,-0.013847,0.101005,0.054280
3,-0.082504,-0.243515,-0.997314,-0.044553,0.179652,0.020891,-0.034164,-0.227783,-0.986816,-0.034155,...,-0.984695,-0.054164,-0.115015,-0.146954,-0.013214,-0.219910,-0.969421,-0.019627,-0.106377,-0.178407
4,0.009186,-0.243484,-0.944733,-0.243911,0.008794,-0.025962,0.005432,-0.237900,-0.974533,-0.253866,...,-1.020767,0.016852,2.405686,-0.119195,0.042725,-0.019501,-1.075653,0.040943,1.520602,-0.205247
5,-0.135345,-0.107437,-0.962555,-0.094749,-0.008850,0.066801,-0.142715,-0.081146,-0.964828,-0.125619,...,-0.678207,-0.123959,-1.271915,0.216070,0.640533,-0.133942,-0.776169,-0.068103,-1.913306,0.249100
6,0.139771,-0.103256,-0.985535,-0.758284,0.033712,-0.010061,0.176605,-0.048859,-1.104172,0.039608,...,-0.510971,0.622647,0.077637,-0.155918,0.146317,-0.386688,-0.442413,-0.147395,0.236832,-0.113768
7,0.134628,-0.010803,-1.321808,0.145257,0.041907,0.131149,0.143921,-0.005508,-1.571655,1.350070,...,-0.371689,-1.046817,-0.341862,0.267625,0.150330,-0.270279,-0.653656,-2.002709,-0.165086,0.092845
8,-0.155197,-0.906769,-0.387787,0.035257,0.020298,0.068753,-0.226685,-0.904037,-0.350311,0.068781,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,-0.431870,-0.690521,-0.619431,-0.024210,0.085979,0.268323,-0.488785,-0.607590,-0.596756,0.213141,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [220]:

rn = RandomForestClassifier()
model = rn.fit(df, y)
joblib.dump(model, 'model.pkl') 


['model.pkl']

In [ ]:
def pad_or_truncate(alist, target_len):
    return alist[:target_len] + [0]*(target_len - len(alist))

In [26]:
payload = r[list(r.keys())[6]]
model = joblib.load("model.pkl") 
x = []
datapoints = payload['datapoints']
for sample in datapoints:
    x.append(sample['ax'])
    x.append(sample['ay'])
    x.append(sample['az'])
    x.append(sample['gx'])
    x.append(sample['gy'])
    x.append(sample['gz'])
x = pad_or_truncate(x, 120)
prediction = model.predict(np.array(x).reshape(1, -1))
predMapped = ymap[(prediction[0]).argmax()]
predMapped

'motion1'

In [51]:
np.array(x).shape

(120,)

In [42]:
a = {"datapoints": [{"ax": 0.1397705078125,
   "ay": -0.1032562255859375,
   "az": -0.98553466796875,
   "gx": -0.7582837213910245,
   "gy": 0.033711624658768036,
   "gz": -0.010060889966101346},
  {"ax": 0.176605224609375,
   "ay": -0.048858642578125,
   "az": -1.1041717529296875,
   "gx": 0.03960786331220347,
   "gy": -0.3446535251048226,
   "gz": 0.14188123865451646},
  {"ax": 0.14544677734375,
   "ay": 0.00042724609375,
   "az": -1.2953033447265625,
   "gx": 0.24784255947437117,
   "gy": 0.4921985084496106,
   "gz": 0.3293416804174121},
  {"ax": 0.108917236328125,
   "ay": -0.093963623046875,
   "az": -1.3127899169921875,
   "gx": 0.9702063630307339,
   "gy": 0.06691724871042101,
   "gz": 0.1667807295823213},
  {"ax": 0.1688690185546875,
   "ay": 0.0322265625,
   "az": -1.1451568603515625,
   "gx": 1.3820823351069007,
   "gy": 0.032121184855772714,
   "gz": 0.08328024308008587},
  {"ax": 0.1215972900390625,
   "ay": -0.009918212890625,
   "az": -0.7676544189453125,
   "gx": 1.1382326530548854,
   "gy": 0.2536916601765122,
   "gz": 0.08423818212418738},
  {"ax": 0.1581268310546875,
   "ay": -0.12982177734375,
   "az": -0.437042236328125,
   "gx": 0.26330380749893517,
   "gy": -0.008794823183877675,
   "gz": 0.16692667081005763},
  {"ax": 0.1516265869140625,
   "ay": -0.1951446533203125,
   "az": -0.428436279296875,
   "gx": -0.6445837870756176,
   "gy": -0.18090560337188358,
   "gz": 0.1448035253186604},
  {"ax": 0.09552001953125,
   "ay": -0.17315673828125,
   "az": -0.5382537841796875,
   "gx": -1.3406462117063578,
   "gy": -0.026706712043532616,
   "gz": 0.004728442515435687},
  {"ax": 0.09869384765625,
   "ay": -0.0757598876953125,
   "az": -0.7368621826171875,
   "gx": -1.1771725955656969,
   "gy": -0.6932533223129247,
   "gz": -0.02676530158751436},
  {"ax": 0.107696533203125,
   "ay": -0.044403076171875,
   "az": -1.02471923828125,
   "gx": -1.4062113735313457,
   "gy": -0.4344675676033089,
   "gz": 0.13005813499510968},
  {"ax": 0.0560302734375,
   "ay": 0.009918212890625,
   "az": -1.3212432861328125,
   "gx": -1.0050708701253974,
   "gy": -0.3097288305695235,
   "gz": 0.09456485556707862},
  {"ax": 0.0600433349609375,
   "ay": 0.0671539306640625,
   "az": -1.4306182861328125,
   "gx": -0.16360277945352975,
   "gy": -0.1282674254781404,
   "gz": 0.1359708852473036},
  {"ax": 0.03900146484375,
   "ay": 0.133636474609375,
   "az": -1.3694000244140625,
   "gx": 0.5423660718000872,
   "gy": 0.10811741603838286,
   "gz": 0.09125800844152723},
  {"ax": 0.0433502197265625,
   "ay": 0.166778564453125,
   "az": -1.3917236328125,
   "gx": 1.3953789657974482,
   "gy": 0.1853333750002493,
   "gz": 0.16893442795586838},
                    {"ax": 0.1397705078125,
   "ay": -0.1032562255859375,
   "az": -0.98553466796875,
   "gx": -0.7582837213910245,
   "gy": 0.033711624658768036,
   "gz": -0.010060889966101346},
  {"ax": 0.176605224609375,
   "ay": -0.048858642578125,
   "az": -1.1041717529296875,
   "gx": 0.03960786331220347,
   "gy": -0.3446535251048226,
   "gz": 0.14188123865451646},
  {"ax": 0.14544677734375,
   "ay": 0.00042724609375,
   "az": -1.2953033447265625,
   "gx": 0.24784255947437117,
   "gy": 0.4921985084496106,
   "gz": 0.3293416804174121},
  {"ax": 0.108917236328125,
   "ay": -0.093963623046875,
   "az": -1.3127899169921875,
   "gx": 0.9702063630307339,
   "gy": 0.06691724871042101,
   "gz": 0.1667807295823213},
  {"ax": 0.1688690185546875,
   "ay": 0.0322265625,
   "az": -1.1451568603515625,
   "gx": 1.3820823351069007,
   "gy": 0.032121184855772714,
   "gz": 0.08328024308008587},
  {"ax": 0.1215972900390625,
   "ay": -0.009918212890625,
   "az": -0.7676544189453125,
   "gx": 1.1382326530548854,
   "gy": 0.2536916601765122,
   "gz": 0.08423818212418738},
  {"ax": 0.1581268310546875,
   "ay": -0.12982177734375,
   "az": -0.437042236328125,
   "gx": 0.26330380749893517,
   "gy": -0.008794823183877675,
   "gz": 0.16692667081005763},
  {"ax": 0.1516265869140625,
   "ay": -0.1951446533203125,
   "az": -0.428436279296875,
   "gx": -0.6445837870756176,
   "gy": -0.18090560337188358,
   "gz": 0.1448035253186604},
  {"ax": 0.09552001953125,
   "ay": -0.17315673828125,
   "az": -0.5382537841796875,
   "gx": -1.3406462117063578,
   "gy": -0.026706712043532616,
   "gz": 0.004728442515435687},
  {"ax": 0.09869384765625,
   "ay": -0.0757598876953125,
   "az": -0.7368621826171875,
   "gx": -1.1771725955656969,
   "gy": -0.6932533223129247,
   "gz": -0.02676530158751436},
  {"ax": 0.107696533203125,
   "ay": -0.044403076171875,
   "az": -1.02471923828125,
   "gx": -1.4062113735313457,
   "gy": -0.4344675676033089,
   "gz": 0.13005813499510968},
  {"ax": 0.0560302734375,
   "ay": 0.009918212890625,
   "az": -1.3212432861328125,
   "gx": -1.0050708701253974,
   "gy": -0.3097288305695235,
   "gz": 0.09456485556707862},
  {"ax": 0.0600433349609375,
   "ay": 0.0671539306640625,
   "az": -1.4306182861328125,
   "gx": -0.16360277945352975,
   "gy": -0.1282674254781404,
   "gz": 0.1359708852473036},
  {"ax": 0.03900146484375,
   "ay": 0.133636474609375,
   "az": -1.3694000244140625,
   "gx": 0.5423660718000872,
   "gy": 0.10811741603838286,
   "gz": 0.09125800844152723},
  {"ax": 0.0433502197265625,
   "ay": 0.166778564453125,
   "az": -1.3917236328125,
   "gx": 1.3953789657974482,
   "gy": 0.1853333750002493,
   "gz": 0.16893442795586838},
  {"ax": 0.0460205078125,
   "ay": 0.0988311767578125,
   "az": -1.3403167724609375,
   "gx": 2.044603843704747,
   "gy": 0.3161502445899226,
   "gz": 0.24048104958917413},
  {"ax": 0.0822296142578125,
   "ay": -0.0576019287109375,
   "az": -0.960723876953125,
   "gx": 1.8443783382048835,
   "gy": 0.2751450206537545,
   "gz": 0.0733533101168155},
  {"ax": 0.08868408203125,
   "ay": -0.1939544677734375,
   "az": -0.6917877197265625,
   "gx": 1.3616497642754857,
   "gy": 0.2523779228107761,
   "gz": -0.10750409003933761},
  {"ax": 0.14129638671875,
   "ay": -0.3482208251953125,
   "az": -0.5109710693359375,
   "gx": 0.6226473291766349,
   "gy": 0.07763673841409896,
   "gz": -0.1559179618119971}
   
 ]}